#ANN

In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [4]:
df = pd.read_csv('query (4).csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-03-18T07:48:59.670Z,24.3636,94.8449,107.717,5.1,mww,74.0,53.0,3.144,0.61,...,2025-03-18T08:18:12.312Z,"83 km ESE of Wāngjing, India",earthquake,9.02,6.868,0.093,11.0,reviewed,us,us
1,2025-03-17T19:37:18.170Z,24.4493,93.4599,44.225,4.1,mb,20.0,149.0,3.497,0.90,...,2025-03-17T20:00:14.040Z,"24 km WNW of Churāchāndpur, India",earthquake,13.24,10.140,0.135,15.0,reviewed,us,us
2,2025-03-14T00:31:26.038Z,27.1365,92.2478,10.000,4.2,mb,42.0,143.0,2.741,0.89,...,2025-03-14T03:02:51.040Z,"22 km SW of Bomdila, India",earthquake,5.74,1.900,0.085,39.0,reviewed,us,us
3,2025-03-13T21:20:11.181Z,33.0777,76.5809,56.996,5.1,mww,77.0,53.0,6.362,0.89,...,2025-03-14T06:37:09.087Z,"51 km SSW of Padam, India",earthquake,9.22,6.437,0.127,6.0,reviewed,us,us
4,2025-03-13T08:37:19.471Z,29.0219,87.5917,10.000,4.3,mb,61.0,101.0,2.373,0.59,...,2025-03-13T12:14:04.040Z,"128 km WSW of Rikaze, China",earthquake,7.64,1.892,0.065,67.0,reviewed,us,us


In [5]:
columns_to_drop = ['net','id','magType', 'nst', 'gap', 'dmin', 'rms', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource']
df = df.drop(columns = columns_to_drop)


In [6]:
df = df.dropna()

In [7]:
df['time'] = pd.to_datetime(df['time'])
df['year'] = df['time'].dt.year.astype('int16')
df['month'] = df['time'].dt.month.astype('int8')
df['Day'] = df['time'].dt.day.astype('int8')
df['hour'] = df['time'].dt.hour.astype('int8')
df = df.drop(columns = ['time'])

In [8]:
df.head()

,latitude,longitude,depth,mag,year,month,Day,hour
0,24.3636,94.8449,107.717,5.1,2025,3,18,7
1,24.4493,93.4599,44.225,4.1,2025,3,17,19
2,27.1365,92.2478,10.000,4.2,2025,3,14,0
3,33.0777,76.5809,56.996,5.1,2025,3,13,21
4,29.0219,87.5917,10.000,4.3,2025,3,13,8


In [9]:
#Optimize data
df['latitude'] = df['latitude'].astype('float32')
df['longitude'] = df ['longitude'].astype('float32')
df['depth'] = df['depth'].astype('float32')
df['mag'] = df['mag'].astype('float32')

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
print("Full DataFrame shape:", df.shape)
print("Columns in DataFrame:", df.columns.tolist())

Full DataFrame shape: (5322, 8)
Columns in DataFrame: ['latitude', 'longitude', 'depth', 'mag', 'year', 'month', 'Day', 'hour']


In [16]:
#setting features and target
features = df[[	'latitude', 'longitude','depth','mag','year','month','Day','hour']]
target = df['mag']

In [17]:
print("Features shape (raw):", features.shape)
print("Target shape (raw):", target.shape)

Features shape (raw): (5322, 8)
Target shape (raw): (5322,)


In [18]:
#scale the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [19]:
#test_train split
x_train, x_test, y_train, y_test = train_test_split(features_scaled, target, test_size = 0.2, random_state = 42)

In [20]:
print("Features shape:", features_scaled.shape)
print("Target shape:", target.shape)

Features shape: (5322, 8)
Target shape: (5322,)


Build ANN

In [26]:
model = Sequential()
model.add(Dense(64, input_dim = x_train.shape[1], activation = 'relu')) 
model.add(Dense(32, activation = 'relu'))
model.add (Dense(1, activation = 'linear'))
model.compile(loss = 'mean_squared_error', optimizer = 'Adam', metrics = ['mae'])

Train

In [29]:
history = model.fit(x_train, y_train, epochs = 50, batch_size = 128, validation_split = 0.2, verbose = 1)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 19.1658 - mae: 4.3090 - val_loss: 7.8852 - val_mae: 2.7243
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.5172 - mae: 2.1857 - val_loss: 1.2594 - val_mae: 0.9281
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0305 - mae: 0.8199 - val_loss: 0.6860 - val_mae: 0.6568
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.6390 - mae: 0.6363 - val_loss: 0.5594 - val_mae: 0.5896
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.5142 - mae: 0.5662 - val_loss: 0.5064 - val_mae: 0.5606
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4786 - mae: 0.5439 - val_loss: 0.4742 - val_mae: 0.5413
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4410 - mae: 0.5236 - val_loss: 0.4426 - val_mae: 0.5227
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4088 - mae: 0.5033 - val_loss: 0.4173 - val_mae: 0.5061
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4016 - mae

Evaluate

In [31]:
loss,mae = model.evaluate(x_test, y_test, verbose = 0)
print(f'Test Mean Absolute Error: {mae:.2f}')

Test Mean Absolute Error: 0.08
